In [1]:
import clip
import torch
from PIL import Image
import json
import requests
import torch.nn.functional as F
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

/home/panyijun/miniconda3/envs/figureClass/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
model, preprocess = clip.load("RN50",device,jit=False)
print(dir(model))

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_call_impl', '_forward_hooks', '_forward_pre_hooks', '_get_backward_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict', '_slow_forward', '_state_dict_hooks', '_version', 'add_module', 'apply', 'bfloat16', 'buffers', 'build_attention_mask', 'childre

In [3]:
def read_json(path):
    with open(path,'r') as f:
        data = json.load(f)
    return data
def get_image_url(url):
    responseImg = requests.get(url)
    if responseImg.status_code == 200:
        #print("Success!")
        image_data = responseImg.content
        with open(f'./tmp.jpg', 'wb') as file:
            file.write(image_data)
            return True
    return False
def get_visual_embed(model,path):
    image = Image.open(path)
    return model.visual(torch.stack([preprocess(image).to(device)]))
def get_text_embed(model,string):
    tokens = clip.tokenize([string[:77]],context_length=77).to(device)
    return model.encode_text(tokens).to(device)

In [13]:
img_paths = read_json("/nfs/turbo/umms-drjieliu/proj/medlineKG/data/figure_json_by_article/pmcimage_paths.json")[:100]
captions = read_json("/nfs/turbo/umms-drjieliu/proj/medlineKG/data/figure_json_by_article/pmcimage_captions.json")[:100]
roco_imgs = read_json("/home/panyijun/PubMed_CLIP/roco_img_paths.json")
roco_captions = read_json("/home/panyijun/PubMed_CLIP/roco_captions.json")


In [5]:
checkpoint = torch.load("./PubMedCLIP_RN50.pth")
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [14]:
def get_similarity(model,url,caption):
    if get_image_url(url):
        print("Loaded successfully")
    visual = get_visual_embed(model,"./tmp.jpg")
    text_embed = get_text_embed(model,caption)
    #print(visual.shape)
    #print(text_embed.shape)
    print(F.cosine_similarity(visual,text_embed,dim=1))

def get_similarity_local(model,path,caption):
    visual = get_visual_embed(model,path)
    text_embed = get_text_embed(model,caption)
    #print(visual.shape)
    #print(text_embed.shape)
    print(F.cosine_similarity(visual,text_embed,dim=1))

In [17]:
for i in range(8):
    get_similarity_local(model,roco_imgs[0],roco_captions[i])


tensor([0.2175], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
tensor([0.1290], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
tensor([0.0888], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
tensor([0.0983], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
tensor([0.1884], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
tensor([0.2184], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
tensor([0.1339], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
tensor([0.1267], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)


In [8]:
model_trained, preprocess = clip.load("RN50",device,jit=False)

In [10]:
!pwd
checkpoint_trained = torch.load("./PubMed_CLIP/checkpoint_test.pth")
model_trained.load_state_dict(checkpoint_trained.state_dict())

/home/panyijun


<All keys matched successfully>

In [11]:
for i in range(8):
    get_similarity(model_trained,img_paths[0],captions[i])

Loaded successfully
tensor([0.2445], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
Loaded successfully
tensor([0.2321], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
Loaded successfully
tensor([0.2455], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
Loaded successfully
tensor([0.2258], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
Loaded successfully
tensor([0.1888], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
Loaded successfully
tensor([0.1584], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
Loaded successfully
tensor([0.1326], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
Loaded successfully
tensor([0.1740], device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)
